In [ ]:
import dicom
import numpy as np
import os
from PIL import Image
import shutil
import scipy.misc
import random
from matplotlib import pyplot as plt
import SimpleITK as sitk
import json
from torchvision import transforms as T
from torchvision.transforms import functional as F
import pandas as pd
import glob
import csv
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

def rm_mkdir(dir_path):
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        print('Remove path - %s'%dir_path)
    os.makedirs(dir_path)
    print('Create path - %s'%dir_path)

In [ ]:
### 數據路徑及分配路徑
#Paths = './B题示例数据/CT影像/'
Paths = '/home/kb/Python_test/Taidi/Image_Segmentation-master/ISIC/dataset/cancer/数据集2/'
list_Paths = os.listdir(Paths)
Dataset = './ISIC/dataset/cancer/Input_pre/'
Dataset_mask = './ISIC/dataset/cancer/Mask_pre/'

In [ ]:
### 裁剪區域
#Loc = (210, 466, 125, 381)
Loc = (170, 490, 100, 420)
# 230-430 189-321

def convertImage(raw):
    img = np.array(raw)
    img[img>255] = 255
    mean = np.mean(img)
    min_img = np.min(img)
    if min_img<-1500:
        min_img = min_img*(1.0/2.0)
    
    img = img - mean
    img = img*1.0 + mean*(mean/min_img)*1.1#修对比度和亮度
    img = img/np.max(img)
    img[img<0] = 0
    return img

def removeImage(dataset, dataset_mask, loc):
    tictoc = 0
    rm_mkdir(dataset)
    rm_mkdir(dataset_mask)
    for i in list_Paths:
        list_PathClasses = os.listdir(Paths+i+'/')
        for j in list_PathClasses:
            list_Dcmfiles = os.listdir(Paths+i+'/'+j+'/')
            negative = 0
            for m in list_Dcmfiles:
                tictoc+=1
                
                if '.dcm' in m:
                    #print(m)
                    image = sitk.ReadImage(Paths+i+'/'+j+'/'+m)
                    image_array = sitk.GetArrayFromImage(image)
                    dcm_image = image_array[0,:,:]
                    convertImg = convertImage(dcm_image)
                    imgRevert = np.repeat(np.expand_dims(convertImg[loc[0]:loc[1],loc[2]:loc[3]],
                                                         axis=2), 3, axis=2)
                    scipy.misc.imsave(dataset+i+'_'+j[0]+'_'+m[:-4]+'.png', imgRevert)
                    
                    tmp_img = np.zeros_like(dcm_image)
                    tmp_maskar = tmp_img[loc[0]:loc[1],loc[2]:loc[3]]
                    tmp_maskar = np.repeat(np.expand_dims(tmp_maskar,
                                                         axis=2), 3, axis=2)

                    scipy.misc.imsave(dataset_mask+i+'_'+j[0]+'_'+m[:-4]+'_mask.png', tmp_maskar)
                if tictoc%1000==0:
                    print('patient is {}, negative samples are {}'.format(i, negative))


In [ ]:
removeImage(Dataset, Dataset_mask, Loc)

cd ISIC

In [ ]:
python ../dataset_cancer.py --train_ratio=0.0\
                                --valid_ratio=0.0\
                                --test_ratio=1.0\
                                --origin_data_path='../ISIC/dataset/cancer/Input_pre'\
                                --origin_GT_path='../ISIC/dataset/cancer/Mask_pre'\
                                --train_path='./dataset/traincancer_pre/'\
                                --train_GT_path='./dataset/traincancer_pre_GT/'\
                                --valid_path='./dataset/validcancer_pre/'\
                                --valid_GT_path='./dataset/validcancer_pre_GT/'\
                                --test_path='./dataset/testcancer_pre/'\
                                --test_GT_path='./dataset/testcancer_pre_GT/'

In [ ]:
python ../main_cancer.py --mode='pre' --model_type='R2AttU_Net'\
                            --testmodel_path='./models/R2AttU_Net-100-0.0000-78-0.4701_re_pretrained.pkl'\
                            --Img_savepath='./Prediction/forwardoutput/'\
                            --Pre_savepath='./Prediction/forwardoutput_GT/'\
                            --Mask_savepath='./Prediction/Prediction/'\
                            --test_path='./dataset/testcancer_pre/' --threshold=90

#### Convert prediction to .json for Mask R-cnn

In [ ]:
Paths_mask_pre = './ISIC/Prediction/forwardoutput_GT/'
print('file length: ',len(os.listdir(Paths_mask_pre)))

In [ ]:
def mask2json(paths):
    tmp_json = {}
    MaskImgs_list = os.listdir(paths)
    for i in MaskImgs_list:
        #print(i[:-4], i)
        tmp_img = Image.open(paths+i).convert('L')
        tmp_img = np.array(tmp_img)
        query = np.where(tmp_img!=0)

        tmp_json[i+str(random.randint(1,10000))] = {'filename' : i[:-4]+'.png',
                                                    'regions' : {'0' : {'shape_attributes' :
                                                                        {'name' : 'ploygon', 
                                                                         'all_points_x' : query[1].tolist(),
                                                                         'all_points_y' : query[0].tolist()}}}}
    return tmp_json

In [ ]:
# val
json_val = mask2json(Paths_mask_pre)
rm_mkdir('./ISIC/Mask_RCNN-master/dataset/Predict/val/')
json.dump(json_val, open('./ISIC/Mask_RCNN-master/dataset/Predict/val/via_region_data.json', 'w'))
! cp -r ./ISIC/Prediction/forwardoutput/* ./ISIC/Mask_RCNN-master/dataset/Predict/val/

#### Input Mask RCNN to prediction

In [ ]:
import os

import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = os.path.abspath("./ISIC/Mask_RCNN-master/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

from samples.balloon import balloon

#os.environ['CUDA_VISIBLE_DEVICES'] = '7'

%matplotlib inline 

MODEL_DIR = os.path.join(ROOT_DIR, "logs")


In [ ]:
config = balloon.BalloonConfig()
BALLOON_DIR = os.path.join(ROOT_DIR, "dataset/Predict")

In [ ]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
#config.display()

In [ ]:
TEST_MODE = "inference"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# Load validation dataset
dataset = balloon.BalloonDataset()
dataset.load_balloon(BALLOON_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

In [ ]:
# Create model in inference mode
#with tf.device(DEVICE):
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

0149.h5 -- 1401
0442.h5 -- 1445.2681875533638 
0262.h5 -- 1470

In [ ]:
# Set path to balloon weights file

# Download file from the Releases page and set its path
# https://github.com/matterport/Mask_RCNN/releases
weights_path = "/home/kb/Python_test/Taidi/Image_Segmentation-master/ISIC/Mask_RCNN-master/"+\
                "logs/balloon20190423T1408/mask_rcnn_balloon_0262.h5"

# Or, load the last model you trained
#weights_path = model.find_last()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

In [ ]:
rm_mkdir('/home/kb/Python_test/Taidi/Image_Segmentation-master/ISIC/Prediction/Prediction_mask_rcnn')

image_id = random.choice(dataset.image_ids)
DC = 0
length = 0
tp = 0
fp = 0
tn = 0
fn = 0 
for image_id in dataset.image_ids:
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)

# Run object detection
    results = model.detect([image], verbose=1)
    r = results[0]

    print(image_id)
    r_shape = r['masks'].shape
    if r_shape[-1]==0:
        SR = np.zeros(shape=(r_shape[0],r_shape[1]), dtype=bool)
    else:
        SR = r['masks'][:,:,0]
    loc = (170, 490, 100, 420)
    #tmp_gt_masks = np.zeros(shape=(520,520))
    tmp_SR = np.zeros(shape=(512,512))
    #tmp_gt_masks[loc[0]:loc[1],loc[2]:loc[3]] = gt_masks
    tmp_SR[loc[0]:loc[1],loc[2]:loc[3]] = SR
    scipy.misc.imsave('/home/kb/Python_test/Taidi/Image_Segmentation-master/ISIC/Prediction/Prediction_mask_rcnn/'+\
                      dataset.image_reference(image_id).split('/')[-1][:-4]+'_mask.png', tmp_SR)

    #print('DC : {}'.format(get_DC(SR, gt_masks)))
    print('-------------------------------------------------------------------')
    length+=1
#DC_average = DC/length

In [ ]:
zerosmask_path = '/home/kb/Python_test/Taidi/Image_Segmentation-master/ISIC/Prediction/Prediction/'
zerosmask_lsit = os.listdir(zerosmask_path)
maskrcnn_path = '/home/kb/Python_test/Taidi/Image_Segmentation-master/ISIC/Prediction/Prediction_mask_rcnn/'
maskrcnn_list = os.listdir(maskrcnn_path)

In [ ]:
B_savepath = '/home/kb/Python_test/Taidi/Image_Segmentation-master/ISIC/Prediction/B_question/'
rm_mkdir(B_savepath)

In [ ]:
aperiod = 'arterial phase'
vperiod = 'venous phase'

In [ ]:
zeros_length = 0
for i in zerosmask_lsit:
    split_list = i.split('_')
    tmp_img = Image.open(zerosmask_path+i)
    if split_list[1]=='a':
        tmp_period = aperiod
    else:
        tmp_period = vperiod
    tmp_path = B_savepath+split_list[0]+'/'+tmp_period+'/'
    if os.path.exists(tmp_path)==False:
        os.makedirs(tmp_path)
    tmp_img.save(tmp_path+split_list[2][:-4]+'_mask.png')
    zeros_length+=1
print('file length is {}'.format(zeros_length))

In [ ]:
mask_length = 0
for i in maskrcnn_list:
    split_list = i.split('_')
    #print(split_list)
    tmp_img = Image.open(maskrcnn_path+i)
    if split_list[1]=='a':
        tmp_period = aperiod
    else:
        tmp_period = vperiod
    tmp_path = B_savepath+split_list[0]+'/'+tmp_period+'/'
    if os.path.exists(tmp_path)==False:
        os.makedirs(tmp_path)
    tmp_img.save(tmp_path+split_list[2]+'_mask.png')
    mask_length+=1
print('file length is {}'.format(mask_length))

In [ ]:
pd.read_excel('/home/kb/Python_test/Taidi/Image_Segmentation-master/ISIC/dataset/cancer/临床数据.xlsx').head(2)

Frame images

In [ ]:
Paths = '/home/kb/Python_test/Taidi/Image_Segmentation-master/ISIC/dataset/cancer/数据集2/'
Mask_Paths = '/home/kb/Python_test/Taidi/Image_Segmentation-master/ISIC/Prediction/B_question/'
frames_save_path = './ISIC/five-video-classification-methods-master/data/cancer/'

list_Paths = os.listdir(Paths)
random.shuffle(list_Paths)

df = pd.read_excel('/home/kb/Python_test/Taidi/Image_Segmentation-master/ISIC/dataset/cancer/临床数据.xlsx')#pd.read_csv('./ISIC/dataset/cancer/B题-全部数据/label2.csv', encoding = "ISO-8859-1")
df = pd.DataFrame(df)
def convertImage(raw):
    img = np.array(raw)
    img[img>255] = 255
    mean = np.mean(img)
    min_img = np.min(img)
    if min_img<-1500:
        min_img = min_img*(1.0/2.0)
    #print(mean, min_img)
    
    img = img - mean
    img = img*1.0 + mean*(mean/min_img)*1.1#修对比度和亮度
    img = img/np.max(img)
    #img[img>255] = 255
    img[img<0] = 0
    return img

In [ ]:
testframes_save_path = './ISIC/five-video-classification-methods-master/data/cancer/test/'
#rm_mkdir(frames_save_path)
rm_mkdir(testframes_save_path+'/negative/')
rm_mkdir(testframes_save_path+'/positive/')
Loc = (170, 490, 100, 420)
def convert_frames(root_paths, mask_root_path, list_root_paths, record, frame_test, loc, ratio = 0.7):
    count = 0
    data_file = []
    tictoc =0 
    for i in list_root_paths:
        tmp_label = 'positive'
        list_period_paths = os.listdir(root_paths+i+'/')
        for j in list_period_paths:
            count+=1
            countmask = 0
            list_patient_paths = os.listdir(root_paths+i+'/'+j+'/')
            dcm_filenames = glob.glob(os.path.join(root_paths+i+'/'+j+'/', '*.dcm'))
            dcm_filenames = sorted(dcm_filenames)
            mask_filenames = glob.glob(os.path.join(mask_root_path+i+'/'+j+'/', '*.png'))
            print()
            mask_filenames = sorted(mask_filenames)
            #for m, pngname in dcm_filenames, mask_filenames:
            for k in range(len(dcm_filenames)):
                m = dcm_filenames[k]
                pngname = mask_filenames[k]
                mask_img = Image.open(pngname).convert('L')
                mask_img = np.array(mask_img)
                if k+1<len(dcm_filenames):
                    
                    pngname_next = mask_filenames[k+1]
                    mask_img_next = Image.open(pngname_next).convert('L')
                    mask_img_next = np.array(mask_img_next)
                else:
                    #pngname_next = mask_filenames[k]
                    #mask_img_next = Image.open(pngname)
                    mask_img_next = mask_img
                if (np.max(mask_img)>0) or (np.max(mask_img_next)>0):
                    countmask+=1
                    tmp_zeros = np.zeros(shape=(512,512,3))
                    #mask_image = np.array(mask_image)
                    image = sitk.ReadImage(m)
                    image_array = sitk.GetArrayFromImage(image)
                    dcm_image = image_array[0,:,:]
                    convertImg = convertImage(dcm_image)
                    #print(pngname, mask_img.shape, convertImg.shape)
                    
                    tmp_zeros[:,:,0] = 0.3*(mask_img[:,:]/255.)*convertImg[:,:] + convertImg[:,:]
                    tmp_zeros[:,:,1] = 0.2*(mask_img[:,:]/255.)*convertImg[:,:] + convertImg[:,:]
                    tmp_zeros[:,:,2] = 0.05*(mask_img[:,:]/255.)*convertImg[:,:] + convertImg[:,:]
                    
                    imgRevert = tmp_zeros[loc[0]:loc[1],loc[2]:loc[3],:]
        
                    #imgRevert = np.repeat(np.expand_dims(convertImg[loc[0]:loc[1],loc[2]:loc[3]],
                    #                                     axis=2), 3, axis=2)
                    m_split = m.split('/')[-1].split('.')[0]

                    scipy.misc.imsave(frame_test+tmp_label+'/'+i+'_'+j[0]+'_au-'+m_split+'.jpg', imgRevert)
            data_file.append(['test', tmp_label, i+'_'+j[0]+'_au', countmask])
           
        print(tmp_label, i, m_split, ' handling process is {}/{}'.format(tictoc, len(list_root_paths)))
        tictoc+=1 
    
    with open(frames_save_path+'data_file_test.csv', 'w') as fout:
        writer = csv.writer(fout)
        writer.writerows(data_file)
    return data_file

In [ ]:
DATA_FILE = convert_frames(Paths, Mask_Paths, list_Paths, df, testframes_save_path, Loc)

In [ ]:
# Root directory of the project
LRCN_DIR = os.path.abspath("./ISIC/five-video-classification-methods-master")

# Import Mask RCNN
sys.path.append(LRCN_DIR)  # To find local version of the library

In [ ]:
"""
Validate our RNN. Basically just runs a validation generator on
about the same number of videos as we have in our test set.
"""
from keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
from models import ResearchModels
from data import DataSet
import os.path
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
sys.path.append('./') 

def validate(data_type, model, seq_length=1, saved_model=None,
             class_limit=None, image_shape=None):
    batch_size = 40

    # Get the data and process it.
    if image_shape is None:
        data = DataSet(
            seq_length=seq_length,
            class_limit=class_limit
        )
    else:
        data = DataSet(
            seq_length=seq_length,
            class_limit=class_limit,
            image_shape=image_shape
        )

    #val_generator = data.frame_generator(batch_size, 'test', data_type)
    X_test, y_test, filenames = data.get_all_sequences_in_memory('test', data_type)

    # Get the model.
    rm = ResearchModels(len(data.classes), model, seq_length, saved_model)

    # Evaluate!
    results = rm.model.predict(
        x=X_test)

    #print(results)
    #print(rm.model.metrics_names)
    return results, y_test, filenames


In [ ]:
#def main():

model = 'lrcn'
saved_model = 'data/checkpoints/lrcn-images.127-0.230.hdf5'

if model == 'conv_3d' or model == 'lrcn':
    data_type = 'images'
    image_shape = (128, 128, 3)
else:
    data_type = 'features'
    image_shape = None
seq_length = 1
pre, y_true, file = validate(data_type, model, saved_model=saved_model,
         image_shape=image_shape, class_limit=4)

# if __name__ == '__main__':
#     main()
